# Nuclear Case Study - Double Loop Workflow

This notebook demonstrates the double loop workflow for the generic nuclear case study

In [1]:
# Import general python packages
from types import ModuleType
from importlib import resources
import pandas as pd

# Imports from the IDAES package
from idaes.apps.grid_integration import Tracker
from idaes.apps.grid_integration import Bidder
from idaes.apps.grid_integration import PlaceHolderForecaster
from idaes.apps.grid_integration import DoubleLoopCoordinator
from idaes.apps.grid_integration.model_data import GeneratorModelData
from idaes.core.solvers import get_solver

# Import Prescient
from prescient.simulator import Prescient

# Import nuclear flowsheet
from nuclear_flowsheet_multiperiod_class import MultiPeriodNuclear

Interactive Python mode detected; using default matplotlib backend for plotting.
    'idaes.core.solvers.get_solver'.  Please update your import. (deprecated
    in 2.0.0.alpha0) (called from c:\Users\Radhakrishna\Anaconda3\lib\site-pac
    kages\dispatches\models\nuclear_case\unit_models\hydrogen_tank_simplified.
    py:24)


In [2]:
generator_data = {
    "gen_name": "121_NUCLEAR_1",
    "generator_type": "thermal",
    "p_min": 400,
    "p_max": 500,
    "min_down_time": 48,
    "min_up_time": 24,
    "ramp_up_60min": 100,
    "ramp_down_60min": 100,
    "shutdown_capacity": 400,
    "startup_capacity": 400,
    "production_cost_bid_pairs": [
        (400, 15),
        (450, 15),
        (500, 15),
    ],
    "startup_cost_pairs": [(48, 7355.42)],
    "fixed_commitment": 1
}

model_data = GeneratorModelData(**generator_data)

In [3]:
tracking_horizon = 12  # hours
bidding_horizon = 48  # hours
n_scenario = 3  # for bidding
n_tracking_hour = 1  # advance n_tracking_hour (i.e. assume we solve every hour)
num_days = 3

In [4]:
with resources.open_text("idaes.apps.grid_integration.examples", "lmp_forecasts_concat.csv") as f:
    forecasts_df = pd.read_csv(f)
forecaster = PlaceHolderForecaster(price_forecasts_df=forecasts_df)

# Create the solver object
default_bid_curve = None
solver = get_solver()

In [5]:
# Setup trackers, bidder, and coordinator
#################################################################################
# Tracker
mp_nuclear_tracker = MultiPeriodNuclear(
    horizon=tracking_horizon,
    model_data=model_data
)
nuclear_tracker = Tracker(
    tracking_model_object=mp_nuclear_tracker,
    n_tracking_hour=n_tracking_hour,
    solver=solver,
)

# Projection Tracker
mp_nuclear_projection_tracker = MultiPeriodNuclear(
    horizon=tracking_horizon,
    model_data=model_data
)
nuclear_projection_tracker = Tracker(
    tracking_model_object=mp_nuclear_projection_tracker,
    n_tracking_hour=n_tracking_hour,
    solver=solver,
)

# Bidder
mp_nuclear_bidder = MultiPeriodNuclear(
    horizon=bidding_horizon,
    model_data=model_data
)
nuclear_bidder = Bidder(
    bidding_model_object=mp_nuclear_bidder,
    n_scenario=n_scenario,
    solver=solver,
    forecaster=forecaster,
)

# Coordinator
coordinator = DoubleLoopCoordinator(
    bidder=nuclear_bidder,
    tracker=nuclear_tracker,
    projection_tracker=nuclear_projection_tracker,
)



2022-06-22 12:51:46 [WARNING] idaes.core.base.unit_model: DEPRECATION: <class 'idaes.core.base.process_block._ScalarElectricalSplitter'> has overloaded the initialize method. In v2.0, IDAES Will be moving to having a centralized initialize method which calls unit-specific initialize_build methods instead. Model developers should update their models to implement the initialize_build method instead of overloading initialize.
2022-06-22 12:51:46 [WARNING] idaes.core.base.unit_model: DEPRECATION: <class 'idaes.core.base.process_block._ScalarPEM_Electrolyzer'> has overloaded the initialize method. In v2.0, IDAES Will be moving to having a centralized initialize method which calls unit-specific initialize_build methods instead. Model developers should update their models to implement the initialize_build method instead of overloading initialize.
2022-06-22 12:51:46 [WARNING] idaes.core.base.unit_model: DEPRECATION: <class 'idaes.core.base.process_block._ScalarSimpleHydrogenTank'> has overloa

In [6]:
class PrescientPluginModule(ModuleType):
    def __init__(self, get_configuration, register_plugins):
        self.get_configuration = get_configuration
        self.register_plugins = register_plugins

In [7]:
plugin_module = PrescientPluginModule(
    get_configuration=coordinator.get_configuration,
    register_plugins=coordinator.register_plugins,
)

In [8]:
rts_gmlc_data_dir = "/Users/Radhakrishna/Desktop/PRESCIENT/downloads/rts_gmlc/RTS-GMLC/RTS_Data/SourceData"

options = {
    "data_path": rts_gmlc_data_dir,
    "input_format": "rts-gmlc",
    "simulate_out_of_sample": True,
    "run_sced_with_persistent_forecast_errors": True,
    "output_directory": "bidding_plugin_test_multiperiod_nulcear",
    "start_date": "2020-07-10",
    "num_days": num_days,
    "sced_horizon": tracking_horizon,
    "ruc_horizon": bidding_horizon,
    "compute_market_settlements": True,
    "day_ahead_pricing": "LMP",
    "ruc_mipgap": 0.01,
    "symbolic_solver_labels": True,
    "reserve_factor": 0.0,
    "deterministic_ruc_solver": "gurobi",
    "output_ruc_solutions": True,
    "sced_solver": "gurobi",
    "print_sced": True,
    "enforce_sced_shutdown_ramprate": True,
    "plugin": {
        "doubleloop": {
            "module": plugin_module,
            "bidding_generator": "121_NUCLEAR_1",
        }
    },
}

Prescient().simulate(**options)

Initializing simulation...


c:\Users\Radhakrishna\Anaconda3\lib\site-packages\pandas\io\parsers.py:3339: FutureWarning: 
        Use pd.to_datetime instead.

  return generic_parser(date_parser, *date_cols)


Setting default t0 state in RTS-GMLC parser
Dates to simulate: 2020-07-10 to 2020-07-12
RUC activation hours: 0
Final RUC date: 2020-07-12
Using current day's forecasts for RUC solves
Using persistent forecast error model when projecting demand and renewables in SCED


Extracting scenario to simulate
Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was com